In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os

import keras
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import load_model

from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

import random as rn
import datetime

In [2]:
# Reading de los modelos.

# Ruta donde están los modelos.

model_CDM = load_model(filepath = "../Versión 3 - actual/modelos/datasetCDM_2023_04_17_19_35")

model_GRD = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRD_2023_04_17_20_53")

model_GRDT = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRDT_2023_04_17_11_29")

model_SEV = load_model(filepath = "../Versión 3 - actual/modelos/datasetSEV_2023_04_17_22_09")

In [3]:
x = pd.read_csv("../../../Datos/Procesados/x_values.csv", dtype = int)
y_cdm = pd.read_csv("../../../Datos/Procesados/y_dummies_cdm.csv", dtype = int)
y_grdt = pd.read_csv("../../../Datos/Procesados/y_dummies_grdtipo.csv", dtype = int)
y_grd = pd.read_csv("../../../Datos/Procesados/y_dummies_grd.csv", dtype = int)
y_sev = pd.read_csv("../../../Datos/Procesados/y_dummies_sev.csv", dtype = int)

In [4]:
y = y_grd
y_str = 'GRD'

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0, stratify = y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 0, stratify = y_train)

In [6]:
y_prediction = model_GRD.predict(x_test)

# score = model_GRDT.evaluate(x_test, y_test, verbose = 1, batch_size = 128)

11900/11900 [==============================] - 24s 2ms/step


In [7]:
y_test_np = y_test.to_numpy(dtype = np.float32)

In [8]:
# class_labels = y_test.columns

In [9]:
# Esto se hizo porque hay una clase que no tiene datos en el recorte, entonces tira problemas el gráficado del recall. (38 si mal no recuerdo)

class_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
       '27', '29', '30', '31', '32', '33', '34', '35', '36', '37', '40',
       '41', '42', '43', '44', '50', '51', '52', '53', '60', '61', '62', '63',
       '64', '65', '70', '71', '72', '80', '81', '82', '83', '84']

In [10]:
y_pred_classes = np.argmax(y_prediction, axis = 1, keepdims = True)
y_true_classes = np.argmax(y_test_np, axis = 1, keepdims = True)

In [11]:
recall_per_class = recall_score(y_true_classes, y_pred_classes, average = None)

In [12]:
len(recall_per_class)

58

In [13]:
def save_recall_plots(recall_per_class, class_labels, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    num_classes = len(class_labels)

    for i in range(num_classes):
        plt.figure(figsize=(6, 4))
        plt.bar(0, recall_per_class[i], color='blue')
        plt.xticks([])
        plt.ylim(0, 1)
        plt.ylabel('Recall')
        plt.title(f'Recall de la Clase {class_labels[i]}')
        plt.savefig(os.path.join(output_folder, f'recall_class_{class_labels[i]}.png'), bbox_inches='tight')
        plt.close()

In [14]:
def save_recall_plots_2(recall_per_class, class_labels, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    num_classes = len(class_labels)

    for i in range(num_classes):
        plt.figure(figsize=(6, 4))
        plt.plot([0, 1], [recall_per_class[i], recall_per_class[i]], color='blue', linewidth=2)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.xticks([])
        plt.ylabel('Recall')
        plt.title(f'Recall de la Clase {class_labels[i]}')
        plt.savefig(os.path.join(output_folder, f'recall_class_{class_labels[i]}.png'), bbox_inches='tight')
        plt.close()


In [15]:
output_folder = f'../../../Visualización de datos/Recall/{y_str}/'

save_recall_plots_2(recall_per_class, class_labels, output_folder)

In [16]:
# y_true_classes son las etiquetas verdaderas convertidas a clases
# y_pred_classes son las etiquetas predichas convertidas a clases

# Calcular el recall promedio ponderado
recall_weighted = recall_score(y_true_classes, y_pred_classes, average='weighted')
print("Recall promedio ponderado:", recall_weighted)

# Calcular el recall promedio macro
recall_macro = recall_score(y_true_classes, y_pred_classes, average='macro')
print("Recall promedio macro:", recall_macro)

# Calcular el recall promedio micro
recall_micro = recall_score(y_true_classes, y_pred_classes, average='micro')
print("Recall promedio micro:", recall_micro)
